In [1]:
library(SingleCellExperiment)
library(ggplot2)
library(multinichenetr)
library(nichenetr)
library(Seurat)
library(tidyverse)
library(data.table)
library(dplyr)
library(rjson)
options(warn=-1)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [2]:
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /home/analysis/miniconda3/envs/scPipe/lib/libopenblasp-r0.3.24.so

locale:
 [1] LC_CTYPE=en_US.utf-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.utf-8        LC_COLLATE=en_US.utf-8    
 [5] LC_MONETARY=en_US.utf-8    LC_MESSAGES=en_US.utf-8   
 [7] LC_PAPER=en_US.utf-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.utf-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] rjson_0.2.21                data.table_1.15.2          
 [3] lubridate_1.9.2             forcats_1.0.0              
 [5] stringr_1.5.1               dplyr_1.1.4                
 [7] purrr_1.0.2                 readr_2.1.4                
 [9] tidyr_1.3.1                 tibble_3.2.

# Download LR database and save to local

In [3]:
# organism = "human"
# options(timeout = 250)

# if(organism == "human"){
  
#   lr_network_all = 
#     readRDS(url(
#       "https://zenodo.org/record/10229222/files/lr_network_human_allInfo_30112033.rds"
#       )) %>% 
#     mutate(
#       ligand = convert_alias_to_symbols(ligand, organism = organism), 
#       receptor = convert_alias_to_symbols(receptor, organism = organism))
  
#   lr_network_all = lr_network_all  %>% 
#     mutate(ligand = make.names(ligand), receptor = make.names(receptor)) 
  
#   lr_network = lr_network_all %>% 
#     distinct(ligand, receptor)
  
#   ligand_target_matrix = readRDS(url(
#     "https://zenodo.org/record/7074291/files/ligand_target_matrix_nsga2r_final.rds"
#     ))
  
#   colnames(ligand_target_matrix) = colnames(ligand_target_matrix) %>% 
#     convert_alias_to_symbols(organism = organism) %>% make.names()
#   rownames(ligand_target_matrix) = rownames(ligand_target_matrix) %>% 
#     convert_alias_to_symbols(organism = organism) %>% make.names()
  
#   lr_network = lr_network %>% filter(ligand %in% colnames(ligand_target_matrix))
#   ligand_target_matrix = ligand_target_matrix[, lr_network$ligand %>% unique()]
  
# } else if(organism == "mouse"){
  
#   lr_network_all = readRDS(url(
#     "https://zenodo.org/record/10229222/files/lr_network_mouse_allInfo_30112033.rds"
#     )) %>% 
#     mutate(
#       ligand = convert_alias_to_symbols(ligand, organism = organism), 
#       receptor = convert_alias_to_symbols(receptor, organism = organism))
  
#   lr_network_all = lr_network_all  %>% 
#     mutate(ligand = make.names(ligand), receptor = make.names(receptor)) 
#   lr_network = lr_network_all %>% 
#     distinct(ligand, receptor)
  
#   ligand_target_matrix = readRDS(url(
#     "https://zenodo.org/record/7074291/files/ligand_target_matrix_nsga2r_final_mouse.rds"
#     ))
  
#   colnames(ligand_target_matrix) = colnames(ligand_target_matrix) %>% 
#     convert_alias_to_symbols(organism = organism) %>% make.names()
#   rownames(ligand_target_matrix) = rownames(ligand_target_matrix) %>% 
#     convert_alias_to_symbols(organism = organism) %>% make.names()
  
#   lr_network = lr_network %>% filter(ligand %in% colnames(ligand_target_matrix))
#   ligand_target_matrix = ligand_target_matrix[, lr_network$ligand %>% unique()]
  
# }

# lr_network_path = '../data/external/lr_network_human_allInfo_30112033.rds'
# ligand_target_matrix_path  = '../data/external/ligand_target_matrix_nsga2r_final.rds'
# saveRDS(lr_network, file=lr_network_path)
# saveRDS(ligand_target_matrix,file=ligand_target_matrix_path)

# Function defination

In [4]:
loadh5ad <- function(inh5ad) {
    library(reticulate)
    ad = import("anndata")
    data_ad = ad$read_h5ad(inh5ad)
    exp.rawdata = data_ad$X
    obs = as.data.frame(data_ad$obs)
    rownames(exp.rawdata) = data_ad$obs_names$to_list()
    colnames(exp.rawdata) = data_ad$var_names$to_list()
    exp.rawdata = t(exp.rawdata)
    return(list(counts=exp.rawdata,obs=obs))
}

getSampleShort <- function(name_list){ 
    sapply(name_list,function(x){paste0('P',paste(strsplit(x,'_')[[1]][c(3,4)],collapse  = '.'))})
           }

replaceChars <- function(name_list){
    name_list = gsub("[.+()-/]", "_", name_list)
    name_list = gsub("0_I", "R", name_list)
    name_list = gsub("II_III", "NR", name_list)
    return(gsub(" ", "_", name_list))
}

# Define parameters

### File path and celltype

In [5]:
COLOR_PAlETTE=fromJSON(file='../data/setting/ColorMap_R_sync.json')

In [6]:
lr_network_path = '../data/external/lr_network_human_allInfo_30112033.rds'
ligand_target_matrix_path  = '../data/external/ligand_target_matrix_nsga2r_final.rds'
sample_id = "Sample_Short"
celltype_id = "Cellstate"
covariates = NA
batches = NA
obs_path = '../data/table/GEX_OBS_Cellstate.csv'
Object_Output_Folder = '../data/object/cell_interaction'
senders_oi = c('Macs_Endocytosis','Macs_Secretion','Macs_Adhesion','Macs_Interferon','Macs_Lipid',
               'Tumor_EMT_III','Tumor_ER_II','Tumor_ER_I','Tumor_Cell_Cycle','Tumor_EMT_II',
               'CD8T_Cytotoxic','CD8T_Naive','CD8T_Exhausted')
receivers_oi =senders_oi

### Parameters for Cell type filtering

### minimum number of cells per cell type per sample at 10 (recommended minimum).

Samples that have less than min_cells cells will be excluded from the analysis for that specific cell type.

In [7]:
min_cells = 10

### Parameters for Gene filtering
For each cell type, we will consider genes expressed if they are expressed in at least a min_sample_prop fraction of samples in the condition with the lowest number of samples. By default, we set min_sample_prop = 0.50.

In [8]:
min_sample_prop = 0.50

For each cell type, we will consider genes expressed if they are expressed in at least a min_sample_prop fraction of samples in the condition with the lowest number of samples. By default, we set min_sample_prop = 0.50.

In [9]:
fraction_cutoff = 0.05 

###  Parameters for ligand activity prediction

We will here choose for applying the p-value cutoff on the normal p-values, and not on the p-values corrected for multiple testing. This choice was made here because this dataset has only a few samples per group and we might have a lack of statistical power due to pseudobulking.

In [10]:
logFC_threshold = 0.50
p_val_threshold = 0.05
p_val_adj = FALSE

After the ligand activity prediction, we will also infer the predicted target genes of these ligands in each contrast. For this ligand-target inference procedure, we also need to select which top n of the predicted target genes will be considered (here: top 250 targets per ligand). This parameter will not affect the ligand activity predictions. It will only affect ligand-target visualizations and construction of the intercellular regulatory network during the downstream analysis. We recommend users to test other settings in case they would be interested in exploring fewer, but more confident target genes, or vice versa.

In [11]:
top_n_target = 250

### Number of cores

In [12]:
cores_system = 8

### Parameters for prioritization

We will use the following criteria to prioritize ligand-receptor interactions:

- Upregulation of the ligand in a sender cell type and/or upregulation of the receptor in a receiver cell type - in the condition of interest.
- Cell-type specific expression of the ligand in the sender cell type and receptor in the receiver cell type in the condition of interest (to mitigate the influence of upregulated but still relatively weakly expressed ligands/receptors).
- Sufficiently high expression levels of ligand and receptor in many samples of the same group.
- High NicheNet ligand activity, to further prioritize ligand-receptor pairs based on their predicted effect of the ligand-receptor interaction on the gene expression in the receiver cell type.
  
We will combine these prioritization criteria in a single aggregated prioritization score. In the default setting, we will weigh each of these criteria equally (scenario = "regular"). This setting is strongly recommended. However, we also provide some additional setting to accomodate different biological scenarios. The setting scenario = "lower_DE" halves the weight for DE criteria and doubles the weight for ligand activity. This is recommended in case your hypothesis is that the differential CCC patterns in your data are less likely to be driven by DE (eg in cases of differential migration into a niche). The setting scenario = "no_frac_LR_expr" ignores the criterion "Sufficiently high expression levels of ligand and receptor in many samples of the same group". This may be interesting for users that have data with a limited number of samples and don’t want to penalize interactions if they are not sufficiently expressed in some samples.

In [13]:
scenario = "regular"

Finally, we still need to make one choice. For NicheNet ligand activity we can choose to prioritize ligands that only induce upregulation of target genes (ligand_activity_down = FALSE) or can lead potentially lead to both up- and downregulation (ligand_activity_down = TRUE). The benefit of ligand_activity_down = FALSE is ease of interpretability: prioritized ligand-receptor pairs will be upregulated in the condition of interest, just like their target genes. ligand_activity_down = TRUE can be harder to interpret because target genes of some interactions may be upregulated in the other conditions compared to the condition of interest. This is harder to interpret, but may help to pick up interactions that can also repress gene expression.

In [14]:
ligand_activity_down = FALSE

# Preprare Data

## Load ligand-receptor network database

In [15]:
lr_network = readRDS(lr_network_path)
ligand_target_matrix = readRDS(ligand_target_matrix_path)

## Celltype annotation data

In [16]:
celltype_col = celltype_id
obs = data.frame(fread(obs_path),row.names=1)
obs = obs %>% 
    mutate(Timepoint=replaceChars(Timepoint)) %>%
    mutate(Treatment_Arm=replaceChars(Treatment_Arm))
obs$Cellstate = replaceChars(obs[,celltype_col])
head(obs)

,Sample_Short,Sample,Compartment,Lineage,Celltype,Cellstate,CCG_ID,BluePrint,AC_Treatment,Tech,WES_Profile,BulkRNA_Profile,PAM50,BestResponse,Timepoint,stage,Patient,Treatment_Arm,br_short
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GTATGTGGTCCTCCAA-1_CCG1112_16_T4_A1_CCG1112_MO_Batch1,P16.T4,CCG1112_16_T4_A1,Immune,Plasma,Plasma,Plasma,CCG1112_16_T4_A1,LumB,Yes,Multiome ATAC + Gene Expression,N,N,N/A,unfavorable response (RCB II-III),Pre_surgery,II,P16,Chemo_>Combo,NR
AACAAGCCAACAGGAT-1_CCG1112_16_T4_A1_CCG1112_MO_Batch1,P16.T4,CCG1112_16_T4_A1,Immune,Myeloid,Macs,Macs_Endocytosis,CCG1112_16_T4_A1,LumB,Yes,Multiome ATAC + Gene Expression,N,N,N/A,unfavorable response (RCB II-III),Pre_surgery,II,P16,Chemo_>Combo,NR
TTATAGCCATATAACC-1_CCG1112_16_T4_A1_CCG1112_MO_Batch1,P16.T4,CCG1112_16_T4_A1,Immune,Myeloid,Macs,Macs_Secretion,CCG1112_16_T4_A1,LumB,Yes,Multiome ATAC + Gene Expression,N,N,N/A,unfavorable response (RCB II-III),Pre_surgery,II,P16,Chemo_>Combo,NR
ATCACAATCCCTCTAA-1_CCG1112_16_T4_A1_CCG1112_MO_Batch1,P16.T4,CCG1112_16_T4_A1,Immune,Myeloid,Macs,Macs_Endocytosis,CCG1112_16_T4_A1,LumB,Yes,Multiome ATAC + Gene Expression,N,N,N/A,unfavorable response (RCB II-III),Pre_surgery,II,P16,Chemo_>Combo,NR
CCTCAAACAGGCATGA-1_CCG1112_16_T4_A1_CCG1112_MO_Batch1,P16.T4,CCG1112_16_T4_A1,Immune,Myeloid,Macs,Macs_Adhesion,CCG1112_16_T4_A1,LumB,Yes,Multiome ATAC + Gene Expression,N,N,N/A,unfavorable response (RCB II-III),Pre_surgery,II,P16,Chemo_>Combo,NR
AGTCAATGTCAATACG-1_CCG1112_16_T4_A1_CCG1112_MO_Batch1,P16.T4,CCG1112_16_T4_A1,Immune,Myeloid,Macs,Macs_Endocytosis,CCG1112_16_T4_A1,LumB,Yes,Multiome ATAC + Gene Expression,N,N,N/A,unfavorable response (RCB II-III),Pre_surgery,II,P16,Chemo_>Combo,NR


## Load GEX h5ad data

In [17]:
h5ad_file = '../data/object/gex_all.h5ad'
data_list = loadh5ad(inh5ad =h5ad_file)
data_list$obs =  data_list$obs %>% 
    tibble::rownames_to_column(var="X") %>%
    left_join(obs %>% tibble::rownames_to_column(var='X'),by='X')%>%
    tibble::column_to_rownames(var='X')

## Create SCE object

In [ ]:
sce_all = SingleCellExperiment(list(counts=data_list$counts),colData=data_list$obs)
sce_all

In [ ]:
sce_all = alias_to_symbol_SCE(sce_all, "human") %>% makenames_SCE()

# Compare R vs NR

In [ ]:
group_id = "br_short"
compared_groups =  c("R","NR")

In [ ]:
contrast = c(paste(compared_groups,collapse ='-'),
  paste(c(compared_groups[2],compared_groups[1]),collapse ='-')
 )
contrasts_oi = c(paste0("'",paste(contrast,collapse ="','"),"'"))
contrast_tbl = tibble(contrast =contrast,
                      group = compared_groups) 
contrasts_oi

## Subset cells that are in senders and receiver cell type

In [ ]:
sce = sce_all[, colData(sce_all)[,celltype_id] %in% c(senders_oi, receivers_oi)]
sce = sce[, colData(sce)[,group_id] %in% compared_groups]
sce

## Perform MultiNicheNet’s cell-cell communication analysis

In [ ]:
n.cores = min(cores_system, union(senders_oi, receivers_oi) %>% length()) # use one core per receiver cell type

#### Celltype-filtering

In [ ]:
abundance_info = get_abundance_info(
  sce = sce, 
  sample_id = sample_id, group_id = group_id, celltype_id = celltype_id, 
  min_cells = min_cells, 
  senders_oi = senders_oi, receivers_oi = receivers_oi, 
  batches = batches
  )
abundance_info$abund_plot_sample

#### Gene filtering

In [ ]:
frq_list = get_frac_exprs(
  sce = sce, 
  sample_id = sample_id, celltype_id =  celltype_id, group_id = group_id, 
  batches = batches, 
  min_cells = min_cells, 
  fraction_cutoff = fraction_cutoff, min_sample_prop = min_sample_prop)

now only keep genes that are expressed by at least one cell type:

In [ ]:
genes_oi = frq_list$expressed_df %>% 
  filter(expressed == TRUE) %>% pull(gene) %>% unique() 
sce = sce[genes_oi, ]

#### Pseudobulk expression calculation
determine and normalize per-sample pseudobulk expression levels for each expressed gene in each present cell type

In [ ]:
abundance_expression_info = process_abundance_expression_info(
  sce = sce, 
  sample_id = sample_id, group_id = group_id, celltype_id = celltype_id, 
  min_cells = min_cells, 
  senders_oi = senders_oi, receivers_oi = receivers_oi, 
  lr_network = lr_network, 
  batches = batches, 
  frq_list = frq_list, 
  abundance_info = abundance_info)

#### Differential expression (DE) analysis
determine which genes are differentially expressed

In [ ]:
DE_info_group1 = get_DE_info(
  sce = sce, 
  sample_id = sample_id, group_id = group_id, celltype_id = celltype_id, 
  batches = batches, covariates = covariates, 
  contrasts_oi = contrasts_oi, 
  min_cells = min_cells, 
  expressed_df = frq_list$expressed_df)

Evaluate the distributions of p-values:

In [ ]:
options(repr.plot.width = 30, repr.plot.height = 10, repr.plot.res = 150)
DE_info_group1$hist_pvals

These p-value distributions look fine, with a uniform distribution with a peak at p-value <= 0.05. 

In [ ]:
empirical_pval = FALSE
if(empirical_pval == TRUE){
  DE_info_emp = get_empirical_pvals(DE_info_group1$celltype_de$de_output_tidy)
  celltype_de_group1 = DE_info_emp$de_output_tidy_emp %>% select(-p_val, -p_adj) %>% 
    rename(p_val = p_emp, p_adj = p_adj_emp)
} else {
  celltype_de_group1 = DE_info_group1$celltype_de$de_output_tidy
} 

#### Combine DE information for ligand-senders and receptors-receivers

In [ ]:
sender_receiver_de = multinichenetr::combine_sender_receiver_de(
  sender_de = celltype_de_group1,
  receiver_de = celltype_de_group1,
  senders_oi = senders_oi,
  receivers_oi = receivers_oi,
  lr_network = lr_network
)

Ligand activity prediction: use the DE analysis output to predict the activity of ligands in receiver cell types and infer their potential target genes

We will first inspect the geneset_oi-vs-background ratios for the default tresholds:

In [ ]:
geneset_assessment = contrast_tbl$contrast %>% 
  lapply(
    process_geneset_data, 
    celltype_de_group1, logFC_threshold, p_val_adj, p_val_threshold
  ) %>% 
  bind_rows() 
geneset_assessment

We can see here that for all cell type / contrast combinations, all geneset/background ratio's are within the recommended range (in_range_up and in_range_down columns). When these geneset/background ratio's would not be within the recommended ranges, we should interpret ligand activity results for these cell types with more caution, or use different thresholds (for these or all cell types).

In [ ]:
ligand_activities_targets_DEgenes = suppressMessages(suppressWarnings(
  get_ligand_activities_targets_DEgenes(
    receiver_de = celltype_de_group1,
    receivers_oi = intersect(receivers_oi, celltype_de_group1$cluster_id %>% unique()),
    ligand_target_matrix = ligand_target_matrix,
    logFC_threshold = logFC_threshold,
    p_val_threshold = p_val_threshold,
    p_val_adj = p_val_adj,
    top_n_target = top_n_target,
    verbose = TRUE, 
    n.cores = n.cores
  )
))

#### Prioritization: rank cell-cell communication patterns through multi-criteria prioritization

In [ ]:
sender_receiver_tbl = sender_receiver_de %>% distinct(sender, receiver)

metadata_combined = SummarizedExperiment::colData(sce) %>% tibble::as_tibble()

if(!is.na(batches)){
  grouping_tbl = metadata_combined[,c(sample_id, group_id, batches)] %>% 
    tibble::as_tibble() %>% distinct()
  colnames(grouping_tbl) = c("sample","group",batches)
} else {
  grouping_tbl = metadata_combined[,c(sample_id, group_id)] %>% 
    tibble::as_tibble() %>% distinct()
  colnames(grouping_tbl) = c("sample","group")
}

prioritization_tables = suppressMessages(multinichenetr::generate_prioritization_tables(
    sender_receiver_info = abundance_expression_info$sender_receiver_info,
    sender_receiver_de = sender_receiver_de,
    ligand_activities_targets_DEgenes = ligand_activities_targets_DEgenes,
    contrast_tbl = contrast_tbl,
    sender_receiver_tbl = sender_receiver_tbl,
    grouping_tbl = grouping_tbl,
    scenario = "regular", # all prioritization criteria will be weighted equally
    fraction_cutoff = fraction_cutoff, 
    abundance_data_receiver = abundance_expression_info$abundance_data_receiver,
    abundance_data_sender = abundance_expression_info$abundance_data_sender,
    ligand_activity_down = ligand_activity_down
  ))

Compile the MultiNicheNet output object

In [ ]:
multinichenet_output_group1_t2vst1 = list(
    celltype_info = abundance_expression_info$celltype_info,
    celltype_de = celltype_de_group1,
    sender_receiver_info = abundance_expression_info$sender_receiver_info,
    sender_receiver_de =  sender_receiver_de,
    ligand_activities_targets_DEgenes = ligand_activities_targets_DEgenes,
    prioritization_tables = prioritization_tables,
    grouping_tbl = grouping_tbl,
    lr_target_prior_cor = tibble()
  ) 
multinichenet_output_group1_t2vst1 = make_lite_output(multinichenet_output_group1_t2vst1)

## Store the result

#### multinichenetR object

In [ ]:
saveRDS(object =multinichenet_output_group1_t2vst1 ,file = file.path(Object_Output_Folder,'multinichenet_NvsNR_Cellstate.rds'))

#### prioritization table

In [ ]:
prioritized_tbl_oi_all = get_top_n_lr_pairs(prioritization_tables, 3000, rank_per_group =T
                                            # senders_oi = c('CD8T','Macs','Tumor'),
                                            # receivers_oi= c('CD8T','Macs','Tumor')
                                           )
write.table(prioritized_tbl_oi_all,'../data/table/multinichenetR/multinichenetr_NR_vs_R_Baseline_Top3000_per_group.tsv',
            row.names =F,quote = F,sep='\t')
write.table(prioritization_tables$sample_prioritization_tbl,
            '../data/table/multinichenetR/multinichenetr_NR_vs_R_Cellstate_prioritization_tbl.tsv',row.names =F,quote = F,sep='\t')
write.table(prioritization_tables$group_prioritization_tbl,
            '../data/table/multinichenetR/multinichenetr_NR_vs_R_Cellstate_prioritization_tbl.tsv',row.names =F,quote = F,sep='\t')

In [ ]:
prioritized_tbl_oi_all = get_top_n_lr_pairs(
  multinichenet_output_group1_t2vst1$prioritization_tables, 
  top_n = 25, 
  rank_per_group = T,
    senders_oi = c('Tumor_EMT_III','Tumor_ER_I','Macs_Secretion','Macs_Adhesion','Macs_Interferon','Macs_Lipid','Macs_Endocytosis'),
    receivers_oi= c('Tumor_EMT_III','Tumor_ER_I','Macs_Secretion','Macs_Adhesion','Macs_Interferon','Macs_Lipid','Macs_Endocytosis'),
  )

prioritized_tbl_oi = 
  multinichenet_output_group1_t2vst1$prioritization_tables$group_prioritization_tbl %>%
  filter(id %in% prioritized_tbl_oi_all$id) %>%
  distinct(id, sender, receiver, ligand, receptor, group) %>% 
  left_join(prioritized_tbl_oi_all)
prioritized_tbl_oi$prioritization_score[is.na(prioritized_tbl_oi$prioritization_score)] = 0

senders_receivers = union(prioritized_tbl_oi$sender %>% unique(), prioritized_tbl_oi$receiver %>% unique()) %>% sort()

colors_sender = c()
colors_name = c()
for( celltype in names(COLOR_PAlETTE$Cellstate)){
    if(celltype %in% senders_receivers){
        colors_sender  = c(colors_sender ,COLOR_PAlETTE$Cellstate[[celltype]])
        colors_name = c(colors_name,celltype)
        }
}
colors_sender = colors_sender %>% magrittr::set_names(colors_name)
colors_receiver = colors_sender

circos_list = make_circos_group_comparison(prioritized_tbl_oi, colors_sender, colors_receiver)
